In [2]:
import nltk
nltk.download()

showing info https://raw.githubusercontent.com/nltk/nltk_data/gh-pages/index.xml


True

In [1]:
import nltk
from nltk.corpus import wordnet as wn
from nltk.corpus import sentiwordnet as swn
from nltk.stem import PorterStemmer
from nltk.tokenize import sent_tokenize
from nltk.tokenize import word_tokenize
from nltk.corpus import stopwords
from nltk.stem.wordnet import WordNetLemmatizer
import pandas as pd
import emoji
import csv
import re
import unicodedata as un
from emosent import get_emoji_sentiment_rank
from nltk.sentiment.vader import SentimentIntensityAnalyzer
sid = SentimentIntensityAnalyzer()
lemmatizer = WordNetLemmatizer()
import flair
flair_sentiment = flair.models.TextClassifier.load('en-sentiment')
import pysentiment2 as ps

hiv4 = ps.HIV4()
lm = ps.LM()

2021-07-01 02:41:51,403 loading file C:\Users\Windows\.flair\models\sentiment-en-mix-distillbert_4.pt


21836 | INFO | NumExpr defaulting to 4 threads.


In [2]:
stop_words=set(stopwords.words("english"))

In [3]:
def extract_emojis(s):
    emojis = []
    text = s
    for c in s:
        try:
            emoji.UNICODE_EMOJI['en'][c]
            emojis.append(c)
            text = text.replace(c,"")
        except:
            None
    return text, emojis

In [4]:
def simplify(text, stop_words):
    text_cl, emojis = extract_emojis(text)
    tokenized_word=word_tokenize(text_cl)
    filtered_sent=[]
    for w in tokenized_word:
        if w not in stop_words:
            filtered_sent.append(w)
    return filtered_sent, emojis

In [5]:
def penn_to_wn(tag):
    """
    Convert between the PennTreebank tags to simple Wordnet tags
    """
    if tag.startswith('J'):
        return wn.ADJ
    elif tag.startswith('N'):
        return wn.NOUN
    elif tag.startswith('R'):
        return wn.ADV
    elif tag.startswith('V'):
        return wn.VERB
    return None



def get_sentiment(word,tag):
    """ returns list of pos neg and objective score. But returns empty list if not present in senti wordnet. """

    wn_tag = penn_to_wn(tag)
    if wn_tag not in (wn.NOUN, wn.ADJ, wn.ADV):
        return []

    lemma = lemmatizer.lemmatize(word, pos=wn_tag)
    if not lemma:
        return []

    synsets = wn.synsets(word, pos=wn_tag)
    if not synsets:
        return []

    # Take the first sense, the most common
    synset = synsets[0]
    swn_synset = swn.senti_synset(synset.name())

    return [swn_synset.pos_score(),swn_synset.neg_score(),swn_synset.obj_score()]

In [6]:
fpath = 'facebook_data_investorsbusinessdaily.tsv'
post_data = pd.read_csv(fpath, sep='\t')
post_data.head()

,ID,PostID,PostDate,Text,Likes,Shares,Comments
0,1.015779e+16,2021-05-31 17:24:31.968425,Trade like the pro's and don't miss this speci...,3,0,2,NaN
1,1.015779e+16,2021-05-31 17:24:33.154690,IBD’s #MemorialDay sale is here! Now’s your ch...,9,0,0,NaN
2,1.015779e+16,2021-05-30 23:24:36.751058,IBD’s #MemorialDay sale is here! Now’s your ch...,11,1,1,NaN
3,1.015779e+16,2021-05-30 16:57:00,IBD’s #MemorialDay sale is here! Now’s your ch...,14,0,1,NaN
4,1.015778e+16,2021-05-30 15:16:00,"Work smarter, not harder. Working Smart Lets...",67,3,3,NaN


In [7]:
post_data.columns = ['PostID','PostDate','Text','Likes','Shares','Comments','Nan']
post_data.head()

,PostID,PostDate,Text,Likes,Shares,Comments,Nan
0,1.015779e+16,2021-05-31 17:24:31.968425,Trade like the pro's and don't miss this speci...,3,0,2,NaN
1,1.015779e+16,2021-05-31 17:24:33.154690,IBD’s #MemorialDay sale is here! Now’s your ch...,9,0,0,NaN
2,1.015779e+16,2021-05-30 23:24:36.751058,IBD’s #MemorialDay sale is here! Now’s your ch...,11,1,1,NaN
3,1.015779e+16,2021-05-30 16:57:00,IBD’s #MemorialDay sale is here! Now’s your ch...,14,0,1,NaN
4,1.015778e+16,2021-05-30 15:16:00,"Work smarter, not harder. Working Smart Lets...",67,3,3,NaN


In [8]:
post_data["ID"] = post_data.index
post_data.head()

,PostID,PostDate,Text,Likes,Shares,Comments,Nan,ID
0,1.015779e+16,2021-05-31 17:24:31.968425,Trade like the pro's and don't miss this speci...,3,0,2,NaN,0
1,1.015779e+16,2021-05-31 17:24:33.154690,IBD’s #MemorialDay sale is here! Now’s your ch...,9,0,0,NaN,1
2,1.015779e+16,2021-05-30 23:24:36.751058,IBD’s #MemorialDay sale is here! Now’s your ch...,11,1,1,NaN,2
3,1.015779e+16,2021-05-30 16:57:00,IBD’s #MemorialDay sale is here! Now’s your ch...,14,0,1,NaN,3
4,1.015778e+16,2021-05-30 15:16:00,"Work smarter, not harder. Working Smart Lets...",67,3,3,NaN,4


In [9]:
#ps = PorterStemmer()
with open('facebook_data_investorsbusinessdaily_post_sentiment_4dict.tsv', 'w', encoding="utf-8") as tsvfile:
    tsv_writer = csv.writer(tsvfile, delimiter='\t')
    tsv_writer.writerow(['ID','PostID','PostDate','Text'
                         , 'Likes', 'Shares'
                         ,'WNPositive','WNNegative', 'WNNeutral','WNSentiment','WNTokenQnt', 'WNSentiAvg'
                         ,'HVPositive','HVNegative','HVTokenQnt', 'HVSentiAvg'
                         ,'LMPositive','LMNegative','LMTokenQnt', 'LMSentiAvg'
                        ,'Vader_Positive', 'Vader_Negative', 'Vader_Neutral','Vader_Compound'
                        ])
    for i in range(post_data.index[0],len(post_data)+post_data.index[0]):
        if not pd.isnull(post_data.loc[i,"Text"]):
            Text = post_data.loc[i,"Text"].replace('#','')
            Text = re.sub(r'^https?:\/\/.*[\r\n]*', '', Text, flags=re.MULTILINE)
            Vader = sid.polarity_scores(Text)
            words_data = simplify(Text, stop_words)[0]
            emoji_data = simplify(Text, stop_words)[1]
            
            HVTokens = hiv4.tokenize(Text)
            HVScoreData = hiv4.get_score(HVTokens)
            
            LMTokens = lm.tokenize(Text)
            LMScoreData = lm.get_score(LMTokens)
            
            # words_data = [ps.stem(x) for x in words_data] # if you want to further stem the word

            pos_val = nltk.pos_tag(words_data)
            senti_val = [get_sentiment(x,y) for (x,y) in pos_val]
            
            emoji_val_cl = []
            if len(emoji_data)>0:
                for emoj in emoji_data:
                    try:
                        emo_dict = get_emoji_sentiment_rank(emoj)
                        emoji_val_cl.append([emo_dict['positive']/emo_dict['occurrences'],
                                            emo_dict['negative']/emo_dict['occurrences'],
                                            emo_dict['neutral']/emo_dict['occurrences']])
                    except:
                        None
                    
            
            positive_score = 0
            negative_score = 0
            objective_score = 0
            
            if len(HVTokens)>0:
                HVPositive = HVScoreData['Positive']
                HVNegative = HVScoreData['Negative']
                HVTokenQnt = len(HVTokens)
                HVSentiAvg = (HVPositive - HVNegative)/HVTokenQnt
            else:
                HVPositive = 0
                HVNegative = 0
                HVTokenQnt = 1
                HVSentiAvg = (HVPositive - HVNegative)/HVTokenQnt
            
            if len(LMTokens)>0:
                LMPositive = LMScoreData['Positive']
                LMNegative = LMScoreData['Negative']
                LMTokenQnt = len(LMTokens)
                LMSentiAvg = (LMPositive - LMNegative)/LMTokenQnt
            else:
                LMPositive = 0
                LMNegative = 0
                LMTokenQnt = 1
                LMSentiAvg = (LMPositive - LMNegative)/LMTokenQnt
            
            senti_val_cl = [val for val in senti_val if val != []]
            
            if len(senti_val_cl)>0:
                for val in senti_val_cl:
                    positive_score+=val[0]
                    negative_score+=val[1]
                    objective_score+=val[2]
            if len(emoji_val_cl)>0:
                for val in emoji_val_cl:
                    positive_score+=val[0]
                    negative_score+=val[1]
                    objective_score+=val[2]
            
            try:
                token_qnt = len(senti_val_cl)+len(emoji_val_cl)
                sentiment_score = positive_score - negative_score
                
                positive_score_avg =positive_score/token_qnt
                negative_score_avg = negative_score/token_qnt
                objective_score_avg = objective_score/token_qnt
                sentiment_score_avg = sentiment_score/token_qnt
                
                 
                

                row = [post_data.loc[i,"ID"],post_data.loc[i,"PostID"],post_data.loc[i,"PostDate"]
                       ,post_data.loc[i,"Text"],post_data.loc[i,"Likes"]
                       ,post_data.loc[i,"Shares"],
                      positive_score, negative_score, objective_score, sentiment_score,token_qnt, sentiment_score_avg
                       ,HVPositive,HVNegative,HVTokenQnt, HVSentiAvg
                       ,LMPositive,LMNegative,LMTokenQnt, LMSentiAvg
                       ,Vader['pos'] , Vader['neg'] , Vader['neu'], Vader['compound']
                      ]
                tsv_writer.writerow(row)
                
            except:
                None
                
        

In [18]:
testtext = """
Yes is actually, all right. 
And i believe it wont decline just that it will be form by less countries than now . It will be reform
"""
testtext = testtext.lower()

words_data = simplify(testtext, stop_words)[0]
            # words_data = [ps.stem(x) for x in words_data] # if you want to further stem the word

pos_val = nltk.pos_tag(words_data)
senti_val = [get_sentiment(x,y) for (x,y) in pos_val]

positive_score = 0
negative_score = 0
objective_score = 0

senti_val_cl = [val for val in senti_val if val != []]


if len(senti_val_cl)>0:
    for val in senti_val_cl:
        positive_score+=val[0]
        negative_score+=val[1]
        objective_score+=val[2]

    positive_score =positive_score/len(senti_val_cl)
    negative_score = negative_score/len(senti_val_cl)
    objective_score = objective_score/len(senti_val_cl)
    sentiment_score = positive_score - negative_score
print([sentiment_score,positive_score, negative_score])

[0.03125, 0.09375, 0.0625]


In [81]:
for i in range(len(pos_val)):
    print([pos_val[i], senti_val[i]])

[('sec0nd', 'JJ'), []]
[('part', 'NN'), [0.0, 0.0, 1.0]]
[('1', 'CD'), []]
[('main', 'JJ'), [0.375, 0.0, 0.625]]
[('texts', 'NN'), [0.0, 0.0, 1.0]]
[('–', 'NNP'), []]
[('sec0nd', 'NN'), []]
[('big', 'JJ'), [0.25, 0.125, 0.625]]
[('buyback', 'NN'), [0.0, 0.0, 1.0]]
[('f0r', 'JJ'), []]
[('irrati0nal', 'JJ'), []]
[('exuberance', 'NN'), [0.5, 0.0, 0.5]]
[('f0ll0wing', 'VBG'), []]
[('secular', 'JJ'), [0.0, 0.0, 1.0]]
[('crisis', 'NN'), [0.25, 0.375, 0.375]]
[('f0r', 'NN'), []]
[('les', 'VBZ'), []]
[('10', 'CD'), []]
[('years', 'NNS'), [0.0, 0.0, 1.0]]
[('eu', 'RB'), []]
[('usa', 'JJ'), []]
[('–', 'NNP'), []]
[('see', 'NN'), [0.0, 0.0, 1.0]]
[('r0bert', 'NN'), []]
[('j.', 'NN'), []]
[('shiller', 'NN'), []]
[('n0bel', 'NN'), []]
[('laureate', 'NN'), [0.5, 0.0, 0.5]]
[('ec0n0mics', 'NNS'), []]
[('f0r', 'VBP'), []]
[('irrati0nal', 'JJ'), []]
[('exuberance', 'NN'), [0.5, 0.0, 0.5]]
[('2009', 'CD'), []]
[('us', 'PRP'), []]
[('p0liticians', 'VBZ'), []]
[('c0nnecti0ns', 'JJ'), []]
[('–', 'NNP'), []

[('m0ney', 'VBP'), []]
[('seri0us', 'NN'), []]
[('crime', 'NN'), [0.0, 0.0, 1.0]]
[('0nly', 'CD'), []]
[('bulg', 'NN'), []]
[('aria', 'NN'), [0.0, 0.0, 1.0]]
[('may', 'MD'), []]
[('mean', 'VB'), []]
[('100', 'CD'), []]
[('th0usand', 'NN'), []]
[('kids', 'NNS'), [0.0, 0.0, 1.0]]
[('trying', 'VBG'), []]
[('drugs', 'NNS'), [0.0, 0.0, 1.0]]
[('.', '.'), []]
[('times', 'NNS'), [0.5, 0.0, 0.5]]
[('0f', 'CD'), []]
[('crisis', 'NN'), [0.25, 0.375, 0.375]]
[('press', 'NN'), [0.0, 0.0, 1.0]]
[('qu0tes', 'VBZ'), []]
[('un', 'JJ'), []]
[('drug', 'NN'), [0.0, 0.0, 1.0]]
[('0ffice', 'CD'), []]
[('vienna', 'NN'), [0.0, 0.0, 1.0]]
[('c0mpanies', 'NNS'), []]
[('turn', 'VBP'), []]
[('t0', 'JJ'), []]
[('dirty', 'NN'), []]
[('business', 'NN'), [0.0, 0.0, 1.0]]
[('t0', 'VBD'), []]
[('0ffset', 'CD'), []]
[('l0sses', 'NNS'), []]
[('.', '.'), []]
[('draghi', 'NN'), []]
[('ju', 'NN'), []]
[('n', 'NN'), [0.0, 0.0, 1.0]]
[('ke', 'NN'), []]
[('r', 'NN'), [0.0, 0.0, 1.0]]
[('trying', 'VBG'), []]
[('anything', 'NN'

In [19]:
get_emoji_sentiment_rank('💩')

{'unicode_codepoint': '0x1f4a9',
 'occurrences': 229,
 'position': 0.839720048,
 'negative': 94.0,
 'neutral': 68.0,
 'positive': 67.0,
 'unicode_name': 'PILE OF POO',
 'unicode_block': 'Miscellaneous Symbols and Pictographs',
 'sentiment_score': -0.118}

In [81]:
testtext = """
ASML CEO Peter Wennink mastered the ability to manage a highly intelligent workforce.  
ASML's 'Authentic' CEO Leads Hundreds Of Ph.D.s To Breakthroughs
"""
get_sentiment(testtext,'n')

[]

In [70]:
hiv4 = ps.HIV4()

In [82]:
tokens = hiv4.tokenize(testtext)  # text can be tokenized by other ways
                                  # however, dict in HIV4 is preprocessed
                                  # by the default tokenizer in the library
score = hiv4.get_score(tokens)

In [83]:
score

{'Positive': 5,
 'Negative': 0,
 'Polarity': 0.99999980000004,
 'Subjectivity': 0.2941176297577865}

In [73]:
(score['Positive'] - score['Negative'])/len(tokens)

-0.2

In [86]:
lm = ps.LM()
tokens = lm.tokenize(testtext)
score = lm.get_score(tokens)

In [87]:
score

{'Positive': 2,
 'Negative': 0,
 'Polarity': 0.99999950000025,
 'Subjectivity': 0.11764705190311459}

In [76]:
(score['Positive'] - score['Negative'])/len(tokens)

-0.2